In [59]:
import pandas as pd
import numpy as np

In [60]:
results = 'results/benchmark_results_25.csv'
times = 'results/embedding_times_25.csv'

In [61]:
df = pd.read_csv(results)
#df.head(10)

In [62]:
grouped_detectors = (
    df.drop_duplicates(subset=['detector', 'split'])  # nos quedamos con una fila por split-detector
      .groupby('detector')
      .agg(total_samples_detected=('n_samples', 'sum'))
      .reset_index()
)

grouped_detectors

,detector,total_samples_detected
0,retinaface,77
1,yolov11n,3
2,yolov11s,6
3,yolov8,7


In [63]:
grouped = df.groupby(['model', 'detector']).agg(
    avg_accuracy=('accuracy', 'mean'),
    total_samples=('n_samples', 'sum')
).reset_index()
grouped.sort_values(by=['total_samples', 'avg_accuracy'], ascending=False, inplace=True)
grouped

,model,detector,avg_accuracy,total_samples
16,VGG-Face,retinaface,0.323772,77
0,ArcFace,retinaface,0.275439,77
4,Facenet,retinaface,0.250775,77
8,GhostFaceNet,retinaface,0.236550,77
12,OpenFace,retinaface,0.208216,77
7,Facenet,yolov8,0.285714,7
19,VGG-Face,yolov8,0.285714,7
11,GhostFaceNet,yolov8,0.142857,7
3,ArcFace,yolov8,0.000000,7
15,OpenFace,yolov8,0.000000,7


In [64]:
detectors_order = [
    "retinaface", "yolov11n", "yolov11s", "yolov8", "centerface", "yunet"
]

models_order = [
    "VGG-Face", "Facenet", "ArcFace", "GhostFaceNet", "OpenFace"
]

# Convertir las columnas a categóricas con el orden deseado
grouped['detector'] = pd.Categorical(grouped['detector'], categories=detectors_order, ordered=True)
grouped['model'] = pd.Categorical(grouped['model'], categories=models_order, ordered=True)

# Ordenar usando ambos niveles
grouped = grouped.sort_values(['detector', 'model']).reset_index(drop=True)
grouped

,model,detector,avg_accuracy,total_samples
0,VGG-Face,retinaface,0.323772,77
1,Facenet,retinaface,0.250775,77
2,ArcFace,retinaface,0.275439,77
3,GhostFaceNet,retinaface,0.236550,77
4,OpenFace,retinaface,0.208216,77
5,VGG-Face,yolov11n,1.000000,3
6,Facenet,yolov11n,1.000000,3
7,ArcFace,yolov11n,0.333333,3
8,GhostFaceNet,yolov11n,1.000000,3
9,OpenFace,yolov11n,1.000000,3


In [65]:
df_times = pd.read_csv(times)
df_times.head(1000)

,model,detector,subject,image_path,embedding_time_sec
0,Facenet,retinaface,Winona_Ryder,/kaggle/working/merged_lfw_cplfw_50_25/Winona_...,0.309709
1,Facenet,retinaface,Winona_Ryder,/kaggle/working/merged_lfw_cplfw_50_25/Winona_...,0.306309
2,Facenet,retinaface,Winona_Ryder,/kaggle/working/merged_lfw_cplfw_50_25/Winona_...,0.302837
3,Facenet,retinaface,Winona_Ryder,/kaggle/working/merged_lfw_cplfw_50_25/Winona_...,0.299131
4,Facenet,retinaface,Winona_Ryder,/kaggle/working/merged_lfw_cplfw_50_25/Winona_...,0.296987
...,...,...,...,...,...
995,Facenet,yolov8,Mohammed_Al-Douri,/kaggle/working/merged_lfw_cplfw_50_25/Mohamme...,0.215545
996,Facenet,yolov8,Mohammed_Al-Douri,/kaggle/working/merged_lfw_cplfw_50_25/Mohamme...,0.209687
997,Facenet,yolov8,Mohammed_Al-Douri,/kaggle/working/merged_lfw_cplfw_50_25/Mohamme...,0.208728
998,Facenet,yolov8,Mohammed_Al-Douri,/kaggle/working/merged_lfw_cplfw_50_25/Mohamme...,0.215658


In [66]:
mask = (df_times['model'] == 'Facenet') & (df['detector'] == 'retinaface')
df_combo = df_times[mask].copy()

# Imputar: identificar el primer registro (por índice o por fila)
# Suponiendo que está en la primera fila con esa combinación:
first_idx = df_combo.index[0]
mean_time = df_combo.loc[df_combo.index != first_idx, 'embedding_time_sec'].mean()
mean_time

np.float64(0.3029232819875081)

In [67]:
df_times.reset_index(drop=True, inplace=True)

# Agrupar por combinación (model, detector)
grouped = df_times.groupby(['model', 'detector'])

# Iterar por cada grupo y reemplazar el primer valor
for (model, detector), group_df in grouped:
    first_idx = group_df.index[0]
    
    # Obtener valor original
    original_value = df_times.loc[first_idx, 'embedding_time_sec']
    
    # Calcular el promedio sin el primer valor
    mean_time = group_df.loc[group_df.index != first_idx, 'embedding_time_sec'].mean()
    
    # Reemplazar en el DataFrame original
    df_times.loc[first_idx, 'embedding_time_sec'] = mean_time
    
    print(f"({model}, {detector}) - Valor original: {original_value:.4f} -> Reemplazado con: {mean_time:.4f}")

(ArcFace, centerface) - Valor original: 0.1280 -> Reemplazado con: 0.1304
(ArcFace, retinaface) - Valor original: 0.1850 -> Reemplazado con: 0.1832
(ArcFace, yolov11n) - Valor original: 0.1050 -> Reemplazado con: 0.1008
(ArcFace, yolov11s) - Valor original: 0.1020 -> Reemplazado con: 0.1024
(ArcFace, yolov8) - Valor original: 0.1255 -> Reemplazado con: 0.0999
(Facenet, centerface) - Valor original: 0.2456 -> Reemplazado con: 0.2441
(Facenet, retinaface) - Valor original: 0.3097 -> Reemplazado con: 0.2940
(Facenet, yolov11n) - Valor original: 0.2183 -> Reemplazado con: 0.2110
(Facenet, yolov11s) - Valor original: 0.2204 -> Reemplazado con: 0.2124
(Facenet, yolov8) - Valor original: 0.2200 -> Reemplazado con: 0.2079
(GhostFaceNet, centerface) - Valor original: 0.2140 -> Reemplazado con: 0.2146
(GhostFaceNet, retinaface) - Valor original: 0.2751 -> Reemplazado con: 0.2668
(GhostFaceNet, yolov11n) - Valor original: 0.1830 -> Reemplazado con: 0.1789
(GhostFaceNet, yolov11s) - Valor original

In [68]:
# Agrupar por model y detector y calcular estadísticas
summary = df_times.groupby(['model', 'detector'])['embedding_time_sec'].agg(
    mean_time='mean',
    min_time='min',
    max_time='max',
    count='count'
).reset_index()

# Ordenar de menor a mayor tiempo promedio
summary_sorted = summary.sort_values(by=['mean_time'], ascending=True)
summary_sorted

,model,detector,mean_time,min_time,max_time,count
24,VGG-Face,yolov8,0.035275,0.031577,0.063385,401
22,VGG-Face,yolov11n,0.035437,0.032028,0.059702,361
23,VGG-Face,yolov11s,0.036030,0.032023,0.054335,421
20,VGG-Face,centerface,0.060772,0.056589,0.079322,232
19,OpenFace,yolov8,0.084820,0.023152,0.120586,401
17,OpenFace,yolov11n,0.086127,0.023793,0.110360,361
18,OpenFace,yolov11s,0.087724,0.024915,0.109308,421
4,ArcFace,yolov8,0.099852,0.029298,0.129716,401
2,ArcFace,yolov11n,0.100832,0.030289,0.125484,361
3,ArcFace,yolov11s,0.102388,0.030968,0.130809,421


In [69]:
summary['detector'] = pd.Categorical(summary['detector'], categories=detectors_order, ordered=True)
summary['model'] = pd.Categorical(summary['model'], categories=models_order, ordered=True)

# Ordena primero por detector, luego por model
summary_sorted = summary.sort_values(['detector', 'model']).reset_index(drop=True)
summary_sorted

,model,detector,mean_time,min_time,max_time,count
0,VGG-Face,retinaface,0.118186,0.112345,0.147288,581
1,Facenet,retinaface,0.293983,0.117067,0.336861,581
2,ArcFace,retinaface,0.183151,0.109303,0.212710,581
3,GhostFaceNet,retinaface,0.266801,0.112391,0.328022,581
4,OpenFace,retinaface,0.169715,0.103538,0.206163,581
5,VGG-Face,yolov11n,0.035437,0.032028,0.059702,361
6,Facenet,yolov11n,0.210956,0.037792,0.256921,361
7,ArcFace,yolov11n,0.100832,0.030289,0.125484,361
8,GhostFaceNet,yolov11n,0.178947,0.031025,0.215960,361
9,OpenFace,yolov11n,0.086127,0.023793,0.110360,361
